# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [17]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import pickle

import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /Users/Joerg 1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Joerg 1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///pipeline_project.db')
df = pd.read_sql_table('messages_merged', 'sqlite:///pipeline_project.db') 
X = df.message.values
Y = df.iloc[:,4:]
categories = Y.columns
Y=Y.values

#https://knowledge.udacity.com/questions/56317
#several 2 values in first column of df > needs to be cleaned
for i, col in enumerate(Y):
    for j, row in enumerate(col):
        if col[j] not in [0, 1]:
            Y[i][j] = 1

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub(r"[^A-Za-z0-9\-]", " ", text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(XGBClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...le_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred=pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(categories)))

                        precision    recall  f1-score   support

               related       0.79      0.98      0.88      4982
               request       0.82      0.51      0.63      1098
                 offer       0.00      0.00      0.00        27
           aid_related       0.79      0.55      0.65      2729
          medical_help       0.69      0.17      0.27       528
      medical_products       0.75      0.25      0.38       320
     search_and_rescue       0.72      0.12      0.21       192
              security       0.33      0.01      0.01       131
              military       0.73      0.20      0.32       215
           child_alone       0.00      0.00      0.00         0
                 water       0.78      0.65      0.71       420
                  food       0.78      0.80      0.79       710
               shelter       0.83      0.51      0.63       604
              clothing       0.74      0.46      0.57       115
                 money       0.79      

/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_f

In [8]:
y_pred=pipeline.predict(X_test)
#print(classification_report(y_test, y_pred, target_names=list(categories)))
labels=np.array(list(categories))
target_names=list(categories)
                
for i in range(len(categories)):
    print(classification_report(y_test[:, i], y_pred[:, i]))


              precision    recall  f1-score   support

           0       0.74      0.20      0.31      1572
           1       0.79      0.98      0.88      4982

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.77      0.59      0.60      6554
weighted avg       0.78      0.79      0.74      6554

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5456
           1       0.82      0.51      0.63      1098

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.86      0.74      0.79      6554
weighted avg       0.89      0.90      0.89      6554

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

   micro avg       1.00      1.00      1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

              preci

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
import os
os.environ["OMP_NUM_THREADS"] = "4"

In [13]:
#https://medium.com/all-things-ai/in-depth-parameter-tuning-for-random-forest-d67bb7e920d
parameters = {
    'clf__estimator__learning_rate': [0.01, 0.2],
    'clf__estimator__max_depth': [10],
    #'clf__estimator__reg_alpha': [0, 4]

}

cv = GridSearchCV(pipeline, cv=5, param_grid=parameters, verbose=1, n_jobs=4)

cv.fit(X_train, y_train)
print(cv.best_score_)
print(cv.best_params_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed: 63.6min finished


NameError: name 'clf' is not defined

In [14]:
print(cv.best_score_)
print(cv.best_params_)

0.28730546231309123
{'clf__estimator__learning_rate': 0.2, 'clf__estimator__max_depth': 10}


In [ ]:
### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred=cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(categories)))

                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      4982
               request       0.78      0.58      0.67      1098
                 offer       0.00      0.00      0.00        27
           aid_related       0.77      0.66      0.71      2729
          medical_help       0.60      0.23      0.33       528
      medical_products       0.67      0.30      0.41       320
     search_and_rescue       0.62      0.15      0.24       192
              security       0.00      0.00      0.00       131
              military       0.66      0.30      0.41       215
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.69      0.72       420
                  food       0.78      0.79      0.79       710
               shelter       0.79      0.60      0.68       604
              clothing       0.74      0.43      0.55       115
                 money       0.62      

/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/python/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_f

### 9. Export your model as a pickle file

In [18]:
model_filepath='classifier.pkl'
with open(model_filepath, 'wb') as f:
        pickle.dump(cv, f)